# example.py in HODLR github

# https://github.com/SAFRAN-LAB/HODLR/blob/master/python/example/example.py

In [4]:
import numpy as np
import pyhodlrlib

# Size of the matrix:
N = 1000
x = np.sort(np.random.rand(N))
print(x.shape)
# Size of leaf level:
M = 200

# Returning the Gaussian Kernel:
class Kernel(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
        if(i == j):
            return 10
        else:
            return np.exp(-(x[i] - x[j])**2)

K = Kernel(N)
# What we are doing here is explicitly generating 
# the matrix from its entries
A = K.getMatrix(0, 0, N, N)
print(A.shape)
# Tolerance for all factorizations:
eps = 1e-12

# If we are assembling a symmetric matrix:
is_sym = True
# If we know that the matrix is also PD:
# By setting the matrix to be symmetric-positive definite, 
# we trigger the fast symmetric factorization method to be used
# In all other cases the fast factorization method is used
is_pd = True
# Creating the HODLR object:
T = pyhodlrlib.HODLR(N, M, eps)
T.assemble(K, 'rookPivoting', is_sym, is_pd)

# Random vector to take product with:
x = np.random.rand(N)
# Finding b using HODLR:
b_hodlr = T.matmatProduct(x)
# Finding b using direct MatVec:
b = A @ x
# Verifying the accuracy of the MatMat:
print('Error in Matrix-Matrix Multiplication:', np.linalg.norm(b_hodlr.ravel() - b.ravel()) / np.linalg.norm(b))

# Factorize elements of the tree:
T.factorize()
# Solving for x in A x = b:
x_hodlr = T.solve(b)
# Computing the relative error:
print('Error in Solve:', np.linalg.norm(x_hodlr.ravel() - x) /  np.linalg.norm(x))

# Finding log determinant:
logdet_hodlr = T.logDeterminant()
# Finding logdet:
logdet = 2 * np.sum(np.log(abs(np.diag(np.linalg.cholesky(A)))))
print('Error in Log-Determinant Computation:', abs(logdet_hodlr - logdet))

# When system described is SPD:
if(is_sym and is_pd):
    # Setting y = W^T x
    y = T.symmetricFactorTransposeProduct(x)
    # Getting b = W (W^T x)
    b_hodlr = T.symmetricFactorProduct(y)
    print('Error in Symmetric Factor Multiplications:', np.linalg.norm(b_hodlr.ravel() - b.ravel()) / np.linalg.norm(b))

    # Directly obtaining the symmetric factor matrix:
    W = T.getSymmetricFactor()
    print('Error in Getting Symmetric Factor:', np.mean(abs(W @ W.T - A)))

(1000,)
(1000, 1000)
Error in Matrix-Matrix Multiplication: 2.2222603314940998e-14
Error in Solve: 1.8242460117049233e-12
Error in Log-Determinant Computation: 0.0
Error in Symmetric Factor Multiplications: 2.212852270003981e-14
Error in Getting Symmetric Factor: 2.48097312693929e-14


# Code https://peterroelants.github.io/posts/gaussian-process-tutorial/

In [10]:

import scipy
from scipy.spatial import distance as dist

In [11]:
# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    """Exponentiated quadratic  with σ=1"""
    # L2 distance (Squared Euclidian)
    sq_norm = -0.5 * scipy.spatial.distance.cdist(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)

In [12]:
# Gaussian process posterior
def GP(X1, y1, X2, kernel_func):
    """
    Calculate the posterior mean and covariance matrix for y2
    based on the corresponding input X2, the observations (y1, X1), 
    and the prior kernel function.
    """
    # Kernel of the observations
    Σ11 = kernel_func(X1, X1)
    # Kernel of observations vs to-predict
    Σ12 = kernel_func(X1, X2)
    # Solve
    solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
    # Compute posterior mean
    μ2 = solved @ y1
    # Compute the posterior covariance
    Σ22 = kernel_func(X2, X2)
    Σ2 = Σ22 - (solved @ Σ12)
    return μ2, Σ2  # mean, covariance

In [13]:
# Compute the posterior mean and covariance

# Define the true function that we want to regress on
f_sin = lambda x: (np.sin(x)).flatten()

n1 = 8  # Number of points to condition on (training points)
n2 = 75  # Number of points in posterior (test points)
ny = 5  # Number of functions that will be sampled from the posterior
domain = (-6, 6)

# Sample observations (X1, y1) on the function
X1 = np.random.uniform(domain[0]+2, domain[1]-2, size=(n1, 1))
y1 = f_sin(X1)
# Predict points at uniform spacing to capture function
X2 = np.linspace(domain[0], domain[1], n2).reshape(-1, 1)
# Compute posterior mean and covariance
μ2, Σ2 = GP(X1, y1, X2, exponentiated_quadratic)
# Compute the standard deviation at the test points to be plotted
σ2 = np.sqrt(np.diag(Σ2))

# Draw some samples of the posterior
y2 = np.random.multivariate_normal(mean=μ2, cov=Σ2, size=1)

# code gpr by changing the function getMatrixEntry

In [5]:
import scipy
from scipy.spatial import distance as dist
class Kernel(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, X, Y):
        
        sq=-0.5*dist.cdist(X, Y, 'sqeuclidean')
        return np.exp(sq)
    def GP_hod(self,X1, y1, X2):
        """
        Calculate the posterior mean and covariance matrix for y2
        based on the corresponding input X2, the observations (y1, X1), 
        and the prior kernel function.
        """
        K = Kernel(N1)
        # What we are doing here is explicitly generating 
        # the matrix from its entries
        A11=K.getMatrixEntry(X1,X1)
        #print(A11,"A11")
       
        # Tolerance for all factorizations:
        eps = 1e-12

        # If we are assembling a symmetric matrix:
        is_sym = True
        # If we know that the matrix is also PD:
        # By setting the matrix to be symmetric-positive definite, 
        # we trigger the fast symmetric factorization method to be used
        # In all other cases the fast factorization method is used
        is_pd = True
        
        # Creating the HODLR object:
        
        #M=3
        #T = pyhodlrlib.HODLR(N1, M, eps)
        #T.assemble(K, 'rookPivoting', is_sym, is_pd)

        #Σ12 = kernel_func(X1, X2)
        # Solve
        A12=K.getMatrixEntry(X1,X2)
        
        solved = scipy.linalg.solve(A11, A12, assume_a='pos').T
        # Compute posterior mean
        μ2 = solved @ y1
        
        
        A22=K.getMatrixEntry(X2,X2)
        
        # Compute the posterior covariance
        Σ2 = A22 - (solved @ A12)
        #print("A11 shape",A11.shape)
        #print("A12 shape",A12.shape)
        #print("A22 shape",A22.shape)
        return μ2, Σ2  # mean, covariance
    # Gaussian process posterior
    def GP(self,X1, y1, X2, kernel_func):
        """
        Calculate the posterior mean and covariance matrix for y2
        based on the corresponding input X2, the observations (y1, X1), 
        and the prior kernel function.
        """
        # Kernel of the observations
        Σ11 = kernel_func(X1, X1)
        # Kernel of observations vs to-predict
        Σ12 = kernel_func(X1, X2)
        # Solve
        solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
       
        # Compute posterior mean
        μ2 = solved @ y1

        # Compute the posterior covariance
        Σ22 = kernel_func(X2, X2)
        
        Σ2 = Σ22 - (solved @ Σ12)

        return μ2, Σ2  # mean, covariance
# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    """Exponentiated quadratic  with σ=1"""
    # L2 distance (Squared Euclidian)
    sq_norm = -0.5 * dist.cdist(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)

In [7]:
# Compute the posterior mean and covariance

# Define the true function that we want to regress on
f_sin = lambda x: (np.sin(x)).flatten()

N1 = 6  # Number of points to condition on (training points)
N2 = 5  # Number of points in posterior (test points)
ny = 5  # Number of functions that will be sampled from the posterior
domain = (-6, 6)

# Sample observations (X1, y1) on the function
X1 = np.random.uniform(domain[0]+2, domain[1]-2, size=(N1, 1))
print(X1.shape)
y1 = f_sin(X1)
# Predict points at uniform spacing to capture function
X2 = np.linspace(domain[0], domain[1], N2).reshape(-1, 1)
# Compute posterior mean and covariance
K = Kernel(N1)
μ2_hod, Σ2_hod = K.GP_hod(X1, y1, X2)
print(μ2_hod, Σ2_hod)
print("******")
μ2, Σ2 = K.GP(X1, y1, X2,exponentiated_quadratic)
print(μ2, Σ2)
#print(Σ2.shape)
# Compute the standard deviation at the test points to be plotted
σ2 = np.sqrt(np.diag(Σ2))
σ2_hod = np.sqrt(np.diag(Σ2_hod))

# Draw some samples of the posterior
y2 = np.random.multivariate_normal(mean=μ2, cov=Σ2, size=1)
y2_hod = np.random.multivariate_normal(mean=μ2_hod, cov=Σ2_hod, size=1)

(6, 1)
[ 0.05573094 -0.03823085  0.1054012   0.01069035 -0.01792854] [[ 9.94545164e-01 -3.81718181e-02  1.21803989e-03 -1.15555062e-04
   9.29957520e-06]
 [-3.81718181e-02  2.10338804e-01 -2.66793506e-02  2.74488515e-03
  -2.21347940e-04]
 [ 1.21803989e-03 -2.66793506e-02  7.11846602e-02 -2.33326049e-02
   2.10988712e-03]
 [-1.15555062e-04  2.74488515e-03 -2.33326049e-02  4.93259306e-02
  -1.51539673e-02]
 [ 9.29957520e-06 -2.21347940e-04  2.10988712e-03 -1.51539673e-02
   9.98600826e-01]]
******
[ 0.05573094 -0.03823085  0.1054012   0.01069035 -0.01792854] [[ 9.94545164e-01 -3.81718181e-02  1.21803989e-03 -1.15555062e-04
   9.29957520e-06]
 [-3.81718181e-02  2.10338804e-01 -2.66793506e-02  2.74488515e-03
  -2.21347940e-04]
 [ 1.21803989e-03 -2.66793506e-02  7.11846602e-02 -2.33326049e-02
   2.10988712e-03]
 [-1.15555062e-04  2.74488515e-03 -2.33326049e-02  4.93259306e-02
  -1.51539673e-02]
 [ 9.29957520e-06 -2.21347940e-04  2.10988712e-03 -1.51539673e-02
   9.98600826e-01]]


# Gpr using HODLR properties

In [9]:
# Gaussian process posterior
# Returning the Gaussian Kernel:
import scipy
from scipy.spatial import distance as dist
class Kernel1(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
         return np.exp(-(X1[i] - X1[j])**2)
    def GP_hod(self,X1, y1, X2):
        # What we are doing here is explicitly generating 
        # the matrix from its entries
        A11=K1.getMatrix(0,0,N1,N1)
        #print(A11,"A11")
        return A11
class Kernel2(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
         return np.exp(-(X1[i] - X2[j])**2)
    def GP_hod(self,X1, y1, X2):
       
        # the matrix from its entries
        A12=K2.getMatrix(0,0,N1,N2)
        #print(A12,"A12")
        
        return A12
class Kernel3(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
         return np.exp(-(X2[i] - X2[j])**2)
    def GP_hod(self,X1, y1, X2):
        
        # the matrix from its entries
        A22=K3.getMatrix(0,0,N2,N2)
        #print(A22,"A22")

        return A22
# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    """Exponentiated quadratic  with σ=1"""
    # L2 distance (Squared Euclidian)
    sq_norm = -0.5 * dist.cdist(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)
 # Gaussian process posterior
def GP(X1, y1, X2, kernel_func):
    """
    Calculate the posterior mean and covariance matrix for y2
    based on the corresponding input X2, the observations (y1, X1), 
    and the prior kernel function.
    """
    # Kernel of the observations
    Σ11 = kernel_func(X1, X1)
    # Kernel of observations vs to-predict
    Σ12 = kernel_func(X1, X2)
    # Solve
    solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
    print("*****")
    print("splved",solved)
    # Compute posterior mean
    μ2 = solved @ y1

    # Compute the posterior covariance
    Σ22 = kernel_func(X2, X2)
    Σ2 = Σ22 - (solved @ Σ12)

    return μ2, Σ2  # mean, covariance

In [10]:
# Compute the posterior mean and covariance

# Define the true function that we want to regress on
f_sin = lambda x: (np.sin(x)).flatten()

N1 = 6  # Number of points to condition on (training points)
N2 = 5  # Number of points in posterior (test points)
#ny = 5  # Number of functions that will be sampled from the posterior
domain = (-6, 6)

# Sample observations (X1, y1) on the function
X1 = np.random.uniform(domain[0]+2, domain[1]-2, size=(N1, 1))
y1 = f_sin(X1)
# Predict points at uniform spacing to capture function
X2 = np.linspace(domain[0], domain[1], N2).reshape(-1, 1)
print(X2.shape)
y2_act=f_sin(X2)
# Compute posterior mean and covariance
K1 = Kernel1(N1)
A11 = K1.GP_hod(X1, y1, X2)
K2 = Kernel2(N1)
A12 = K2.GP_hod(X1, y1, X2)
K3 = Kernel3(N2)
A22 = K3.GP_hod(X1, y1, X2)
eps = 1e-12
# If we are assembling a symmetric matrix:
is_sym = True
# If we know that the matrix is also PD:
# By setting the matrix to be symmetric-positive definite, 
# we trigger the fast symmetric factorization method to be used
# In all other cases the fast factorization method is used
is_pd = True
# Creating the HODLR object:
#print(type(A11))
M=3
T = pyhodlrlib.HODLR(N1, M, eps)
T.assemble(K1, 'rookPivoting', is_sym, is_pd)
# Factorize elements of the tree:
T.factorize()
# Solving for x in A x = b:
x_hodlr = T.solve(A12)
x_hodlr=x_hodlr.T
# Compute posterior mean
#print(x_hodlr.shape,y1.shape,A12.shape,A22.shape)
μ2_hod = x_hodlr @ y1
Σ2_hod = A22 - (x_hodlr @ A12)
print("22",μ2_hod, Σ2_hod,"22")
print("******")
μ2, Σ2 = GP(X1, y1, X2,exponentiated_quadratic)
print("11",μ2, Σ2,"11")
# Compute the standard deviation at the test points to be plotted
σ2 = np.sqrt(np.diag(Σ2))
σ2_hod = np.sqrt(np.diag(Σ2_hod))

# Draw some samples of the posterior
y2 = np.random.multivariate_normal(mean=μ2, cov=Σ2, size=1)
y2_hod = np.random.multivariate_normal(mean=μ2_hod, cov=Σ2_hod, size=1)

(5, 1)
22 [-1.85981932e-06 -2.99862641e-01 -4.24386088e-02 -2.85644676e-01
 -2.53423962e-03] [[ 1.00000000e+00  1.14030332e-04  5.80348949e-07 -8.04957694e-09
   2.12595993e-12]
 [ 1.14030332e-04  1.81702809e-01  1.87155642e-02 -2.68008607e-04
   7.07844638e-08]
 [ 5.80348949e-07  1.87155642e-02  7.68102283e-02 -4.31679155e-03
   1.15435664e-06]
 [-8.04957694e-09 -2.68008607e-04 -4.31679155e-03  6.13457644e-01
  -2.87574981e-03]
 [ 2.12595993e-12  7.07844638e-08  1.15435664e-06 -2.87574981e-03
   9.99976626e-01]] 22
******
*****
splved [[ 1.51272302e-02  5.14897542e-02 -6.01228935e-02 -4.08226681e-05
  -3.66599740e-03  9.12006910e-04]
 [ 1.65142483e+00  2.38412548e+00 -3.02257451e+00 -1.38136336e-03
  -1.30664246e-01  3.09173144e-02]
 [-1.36018299e-01 -1.22009258e+00  1.16669560e+00 -4.73767762e-03
   1.07819470e+00  1.15293086e-01]
 [ 5.89477454e-02  4.46038939e-01 -4.46789286e-01  7.78328080e-01
  -1.13122257e-01  2.02001484e-01]
 [-1.24089704e-03 -9.35867272e-03  9.38208854e-03  6.9

In [53]:
y2[0]

array([-0.12930635, -0.06712564, -0.46085581,  0.1889964 , -0.35303797])

In [55]:
y2_hod[0]

array([-1.18604743,  0.01075804,  1.31618782,  0.53153043, -0.34720879])

In [56]:
(y2_act)

array([ 0.2794155 , -0.14112001,  0.        ,  0.14112001, -0.2794155 ])

In [57]:
x_hodlr

array([[ 1.03577798e-09, -2.54642497e-05,  2.01713344e-08,
         6.52278081e-05, -1.38124279e-08, -9.92893477e-10],
       [ 4.11122988e-06, -1.01786662e-01,  8.00717712e-05,
         1.02279311e+00, -5.48249143e-05, -3.94101166e-06],
       [ 2.97201636e-02,  3.14437305e-02,  7.34448243e-01,
        -1.20337228e-02, -4.03840130e-01, -2.84850523e-02],
       [ 4.34973173e+00,  5.02301966e-05, -1.06560576e-01,
        -1.95711344e-05,  1.68965012e-01, -3.89224749e+00],
       [-2.44653194e-01,  2.02870160e-07, -4.29343478e-04,
        -7.90445885e-08,  6.40178015e-04,  2.56440808e-01]])

In [59]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y2_act,y2_hod[0])
print('MAPE score is {}'.format(mape))

MAPE score is 1185516592564450.0


In [60]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y2_act,y2[0])
print('MAPE score is {}'.format(mape))

MAPE score is 415102010526475.7
